In [1]:
import re
from typing import Dict, Optional, List
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import numpy as np

class EcoMarketAsistenteRAGEmbeddings:
    def __init__(self):
        # -------------------- Base de datos simulada --------------------
        self.pedidos_db = {
            "12345": {"nombre": "María García", "estado": "en tránsito", "carrier": "DHL Express", "fecha": "2025-09-26"},
            "12346": {"nombre": "Juan Pérez", "estado": "procesando", "carrier": "Servientrega", "fecha": "2025-09-28"},
            "12347": {"nombre": "Ana López", "estado": "entregado", "carrier": "Coordinadora", "fecha": "2025-09-24"},
        }

        # Documentos adicionales (para RAG: políticas, FAQs, etc.)
        self.documentos = [
            "Las devoluciones se aceptan hasta 30 días después de la compra.",
            "Los pedidos se entregan generalmente en un plazo de 3 a 5 días hábiles.",
            "Los clientes pueden solicitar un reembolso completo, crédito en tienda o reemplazo.",
            "El pedido en tránsito significa que está en camino con la transportadora.",
            "EcoMarket trabaja con DHL Express, Servientrega y Coordinadora como carriers principales."
        ]

        # -------------------- Embeddings (SentenceTransformers) --------------------
        self.embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
        self.doc_embeddings = self.embedder.encode(self.documentos, convert_to_tensor=False)

        # Guardar último pedido mencionado
        self.pedido_actual = None

        # -------------------- Modelo Hugging Face --------------------
        self.generator = pipeline(
            "text2text-generation",
            model="google/flan-t5-small"   # Modelo pequeño y factual
        )

        # Plantilla base
        self.prompt_base = """Eres un asistente de EcoMarket.
Tu tarea es dar respuestas breves, claras y verificables
sobre pedidos y productos. Nunca inventes información, usa solo lo que se te proporcione.

Datos del cliente:
- Nombre: {nombre}
- Pedido: {id_pedido} → Estado: {estado}, Carrier: {carrier}, ETA: {fecha}

Contexto adicional:
{contexto}

Responde con tono amable y profesional.

Cliente: {mensaje_cliente}
Respuesta:
"""

    # -------------------- Funciones auxiliares --------------------
    def extraer_numero_pedido(self, mensaje: str) -> Optional[str]:
        patrones = [r'#(\d+)', r'pedido\s*(\d+)', r'orden\s*(\d+)']
        for patron in patrones:
            match = re.search(patron, mensaje.lower())
            if match:
                return match.group(1)
        return None

    def obtener_datos_pedido(self, id_pedido: str) -> Optional[Dict]:
        return self.pedidos_db.get(id_pedido)

    def recuperar_contexto(self, consulta: str, top_k: int = 2) -> List[str]:
        """Busca documentos relevantes usando embeddings semánticos"""
        consulta_emb = self.embedder.encode([consulta], convert_to_tensor=False)[0]
        similitudes = np.dot(self.doc_embeddings, consulta_emb) / (
            np.linalg.norm(self.doc_embeddings, axis=1) * np.linalg.norm(consulta_emb)
        )
        indices = np.argsort(similitudes)[-top_k:][::-1]
        return [self.documentos[i] for i in indices]

    # -------------------- Generar respuesta --------------------
    def generar_respuesta(self, mensaje_cliente: str) -> str:
        id_pedido = self.extraer_numero_pedido(mensaje_cliente)

        if id_pedido:
            self.pedido_actual = id_pedido
        elif not self.pedido_actual:
            return "Hola, para ayudarte necesito el número de pedido. Por favor proporciónalo."
        else:
            id_pedido = self.pedido_actual

        datos = self.obtener_datos_pedido(id_pedido)
        if not datos:
            return f"No encontré información del pedido #{id_pedido}. Verifica el número."

        # Recuperar contexto adicional (RAG con embeddings)
        contexto = "\n".join(self.recuperar_contexto(mensaje_cliente))

        # Construir prompt factual con RAG
        prompt = self.prompt_base.format(
            nombre=datos["nombre"],
            id_pedido=id_pedido,
            estado=datos["estado"],
            carrier=datos["carrier"],
            fecha=datos["fecha"],
            contexto=contexto,
            mensaje_cliente=mensaje_cliente
        )

        # Generar con FLAN-T5
        salida = self.generator(prompt, max_new_tokens=80)[0]["generated_text"]
        return salida.strip()


# -------------------- PRUEBA --------------------
if __name__ == "__main__":
    asistente = EcoMarketAsistenteRAGEmbeddings()
    casos = [
        "¿Dónde está mi pedido #12345?",
        "Quiero saber cómo funcionan las devoluciones",
        "Mi pedido 12346 no ha llegado aún",
        "¿Qué transportadoras trabajan con ustedes?"
    ]

    for caso in casos:
        print(f"Cliente: {caso}")
        print(f"Asistente: {asistente.generar_respuesta(caso)}\n")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Cliente: ¿Dónde está mi pedido #12345?
Asistente: Estado: en tránsito, Carrier: DHL Express, ETA: 2025-09-26

Cliente: Quiero saber cómo funcionan las devoluciones
Asistente: Eres a asistente de EcoMarket. Tu tarea es dar respuestas breves, claras y verificables sobre pedidos y productos. Nunca inventes información, usa solo lo que se te proporcione. Datos del cliente:

Cliente: Mi pedido 12346 no ha llegado aún
Asistente: Eres un asistente de EcoMarket. Tu tarea es dar respuestas breves, claras y verificables sobre pedidos y productos. Nunca inventes información, usa solo lo que se te proporcione. Datos del cliente:

Cliente: ¿Qué transportadoras trabajan con ustedes?
Asistente: EcoMarket trabaja con DHL Express, Servientrega y Coordinadora como carriers principales. Responde con tono amable y profesional. Cliente: Qué transportadoras trabajan con ustedes?

